In [12]:
# 1. Importación de librerías

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder


In [13]:
# 2. Cargar logs ordenados por tiempo
df = pd.read_csv("C:/Users/alons/Desktop/Tesis_SIEM_ML/data/processed/logs_unificados_limpios.csv")
df = df.sort_values(by='@timestamp')

In [14]:
# 3. Crear pares de eventos consecutivos: (evento_actual, evento_siguiente)
secuencias = []
prev_event = None

for event in df['danger.type']:
    if prev_event is not None:
        secuencias.append((prev_event, event))
    prev_event = event

In [15]:
# 4. Codificar los eventos y guardar el encoder
le = LabelEncoder()
le.fit(df['danger.type'])
df['danger_encoded'] = le.transform(df['danger.type'])

# Guardar el encoder
joblib.dump(le, "C:/Users/alons/Desktop/Tesis_SIEM_ML/models/label_encoder_danger_type.pkl")

['C:/Users/alons/Desktop/Tesis_SIEM_ML/models/label_encoder_danger_type.pkl']

In [16]:
# 5. Codificar las secuencias
secuencias_codificadas = [(le.transform([a])[0], le.transform([b])[0]) for a, b in secuencias]

In [17]:
# 6. Construir una matriz de transición de eventos
transiciones = defaultdict(lambda: defaultdict(int))

for a, b in secuencias_codificadas:
    transiciones[a][b] += 1

In [18]:
# 7. Calcular probabilidades de transición
trans_df = pd.DataFrame(transiciones).T.fillna(0).astype(int)
trans_df_prob = trans_df.div(trans_df.sum(axis=1), axis=0)

In [19]:
# 8. Guardar modelo de transiciones
trans_df_prob.to_csv("C:/Users/alons/Desktop/Tesis_SIEM_ML/models/matriz_transiciones.csv")
print("Matriz de transiciones guardada.")

Matriz de transiciones guardada.


In [40]:
# Cargar el DataFrame original para obtener los nombres correctos
df_original = pd.read_csv("C:/Users/alons/Desktop/Tesis_SIEM_ML/data/processed/logs_unificados_limpios.csv")

# Crear mapeo de códigos a nombres
le = joblib.load("C:/Users/alons/Desktop/Tesis_SIEM_ML/models/label_encoder_danger_type.pkl")
codigo_a_nombre = dict(zip(le.transform(df_original['danger.type']), df_original['danger.type']))


In [41]:
# 9. Función para predecir la siguiente acción más probable dado un evento
def predecir_siguiente_nombre(evento_codificado, top_n=3):
    if evento_codificado in trans_df_prob.index:
        predicciones = trans_df_prob.loc[evento_codificado].sort_values(ascending=False).head(top_n)

        resultado = {}
        for idx, prob in predicciones.items():
            clave = int(idx)  # conversión explícita
            nombre = codigo_a_nombre.get(clave, f'Clase {clave}')
            resultado[nombre] = round(float(prob), 4)

        return resultado
    else:
        return "Evento no observado anteriormente"
    
# 10. Ejemplo: predecir qué viene después del evento tipo '39'
print("Predicción de acciones siguientes al evento '39':")
print(predecir_siguiente_nombre(39))

Predicción de acciones siguientes al evento '39':
{39: 0.6934, 0: 0.2602, 45: 0.027}
